# Egocentric Video Mapper

Before starting make sure you have an available GPU. If you are unsure about whether you have an available GPU or if you want to check which GPU you will be working with run the next executble cell.

In [1]:
!nvidia-smi -L

GPU 0: NVIDIA GeForce GTX 1080 Ti (UUID: GPU-72b7ddec-7074-1583-0076-9281c065ae9d)


Now run the next cell to import the modules and functions that will be used for this AlphaLab

In [1]:
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from utils import (
    VideoHandler,
    write_timestamp_csv,
    generate_mapper_kwargs,
    generate_comparison_video_kwargs,
)
from optic_flow import OpticFlowCalculatorLK
from sync_videos import OffsetCalculator
from gaze_mapper import RulesBasedGazeMapper
from video_renderer import save_comparison_video, save_gaze_video

Fill out the path to the directory of the uncompressed Scene Video + Timeseries download from Cloud.

In [2]:
neon_timeseries_path = (
    "/users/sof/video_examples/second_video/2024-05-23_16-47-35-a666ea62"
)
neon_vid_path = Path(neon_timeseries_path).rglob("*.mp4").__next__()

Fill out the path of the corresponding alternative egocentric view recording (please make sure that both videos have the same orientation).

In [3]:
alternative_vid_path = "/users/sof/video_examples/second_video/20240523_171941_000.mp4"

Define an output directory where the different output files will be saved.

In [4]:
output_dir = "/users/sof/action_map_experiments/s_try"

## Step 1: Video Synchronization

In this first step, optic flow in both videos will be calculated. Each optic flow result is saved to a csv file in an automatically created optic_flow subdirectory in the output directory. The csv file will contain the following columns:
- start: the start frame timestamp of the optic flow calculation
- end: the end frame timestamp of the optic flow calculation
- dx: Average horizontal displacement  
- dy: Average vertical displacement 
- angle: Average angular change in degrees

Optic flow is then used to obtain the time offset of the Action video with respect to the Neon Scene video. 



In [5]:
neon_of = OpticFlowCalculatorLK(video_dir=neon_vid_path)
neon_of_result = neon_of.process_video(
    output_file=Path(output_dir, "optic_flow/neon_lk_of.csv")
)

alternative_of = OpticFlowCalculatorLK(video_dir=alternative_vid_path)
alternative_of_result = alternative_of.process_video(
    output_file=Path(output_dir, "optic_flow/alternative_lk_of.csv")
)

Once optic flow is measured, the time offset can be calculated and a action_worldtimestamps.csv will be generated with a similar format as Neon's world_timestamps.csv.

In [6]:
offset_calc = OffsetCalculator(
    src=alternative_of_result["dy"].values,
    src_timestamps=alternative_of_result["start"].values,
    dst=neon_of_result["dy"].values,
    dst_timestamps=neon_of_result["start"].values,
    resampling_frequency=500,
)

t_offset, pearson_corr = offset_calc.estimate_time_offset()
print(
    f"Estimated offset of alternative video with respect to Neon scene video: {t_offset} seconds (Pearson correlation: {pearson_corr})"
)

write_timestamp_csv(
    neon_timeseries_path,
    VideoHandler(alternative_vid_path).timestamps + t_offset,
    saving_path=output_dir,
)

Estimated offset of alternative video with respect to Neon scene video: 2.016 seconds (Pearson correlation: 0.9192123717590794)


Last timestamp of alternative camera recording (2024-05-23 14:48:20.852368384) is after the last timestamp of Neon Scene recording (2024-05-23 14:48:18.721666666)


## Step 2: Map gaze data to the action video

After synchronizing both videos, it is time to obtain Neon gaze signal in the coordinate system of the other video. 

Here we make use of deep learning methods to guide the gaze transformation, the deafault in this notebook is uses an implementation of Efficient LOFTR.

We have implemented other algorithms: 'LOFTR', 'DISK_LightGlue', 'DeDoDe_LightGlue'. If you wish to try them out, write the name of the desired algorithm in the `image_matcher_choice` variable in the cell below.

In [7]:
image_matcher_choice = "Efficient_LOFTR"  # Options: 'Efficient_LOFTR', 'LOFTR', 'DISK_LightGlue', 'DeDoDe_LightGlue'

mapper_kwargs = generate_mapper_kwargs(
    neon_timeseries_path, alternative_vid_path, output_dir, image_matcher_choice
)

mapper = RulesBasedGazeMapper(**mapper_kwargs)

Before running the mapping pipeline you can adjust how often (in seconds) you want the image correspondances refreshed. 

- The higher you set the time threshold the less time the mapping will take however it won't be as accurate as it could be. 

- The lower the time threshold the more time the mapping will take. 

If you leave the value at 'None' then for every gaze new imge correspondences will be obtained. We recommend using values smaller than 1 second.

In [8]:
time_threshold = 0.5

In [ ]:
mapping_kwargs = {
    "saving_path": Path(
        output_dir,
        f"mapped_gaze/{image_matcher_choice.lower()}/alternative_camera_gaze_lk.csv",
    ),
    "refresh_time_thrshld": time_threshold,
}
mapped_gaze_path = mapper.map_gaze(**mapping_kwargs)

## Step 3 (OPTIONAL): Create visualization videos

For visualization purposes you can get a side-to-side rendering with gaze of the Neon Scene video and the alternative egocentric video.  


In [ ]:
comparison_video_kwargs = generate_comparison_video_kwargs(
    neon_timeseries_path,
    alternative_vid_path,
    mapped_gaze_path,
    output_dir,
    image_matcher_choice,
)
save_comparison_video(**comparison_video_kwargs)

Additionally, you can get only the alternative egocentric video (at its original frame rate) with the overlaid gaze circle.

In [ ]:
gaze_video_args = {
    "video_path": alternative_vid_path,
    "timestamps_path": Path(output_dir, "action_camera_timestamps.csv"),
    "gaze_path": Path(mapped_gaze_path),
    "save_video_path": Path(
        output_dir, f"rendered_videos/{image_matcher_choice}_lk.avi"
    ),
}

save_gaze_video(**gaze_video_args)